<a href="https://colab.research.google.com/github/Tonry12/Data_joyx2/blob/main/Lottery_Prediction_model_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import os
import pickle

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
lotto = pd.read_csv('/content/drive/MyDrive/SPN/Predictive lottery.xlsx - since 30 years ago.csv')

In [ ]:
lotto = lotto.drop(columns=['Date','Unnamed: 1','Unnamed: 2','Unnamed: 3'])

In [ ]:
lotto

,DATE,รางวัลที่ 1,2 ตัวบน,3 ตัวบน,2 ตัวล่าง,3 ตัวหน้า,3 ตัวหน้า.1,3 ตัวล่าง,3 ตัวล่าง.1,weekday_dum,weekday
0,5/1/2024,980116,16,116,17,104,763,634,833,4,Wednesday
1,4/16/2024,943598,98,598,79,727,729,154,200,3,Tuesday
2,4/1/2024,803481,81,481,90,122,809,559,947,2,Monday
3,3/16/2024,997626,26,626,78,509,571,329,794,7,Saturday
4,3/1/2024,253603,3,603,79,900,975,382,703,6,Friday
...,...,...,...,...,...,...,...,...,...,...,...
716,5/16/1994,2071764,64,764,29,93,544,535,746,2,Monday
717,5/2/1994,3072877,77,877,69,535,317,170,521,2,Monday
718,4/16/1994,7893913,13,913,21,785,982,151,594,7,Saturday
719,4/1/1994,4269815,15,815,29,675,35,704,868,6,Friday


In [ ]:
lotto.isnull().any()

DATE           False
รางวัลที่ 1    False
2 ตัวบน        False
3 ตัวบน        False
2 ตัวล่าง      False
3 ตัวหน้า      False
3 ตัวหน้า.1    False
3 ตัวล่าง      False
3 ตัวล่าง.1    False
weekday_dum    False
weekday        False
dtype: bool

In [ ]:
lotto.rename(columns={'DATE': 'Date','รางวัลที่ 1': '1st','3 ตัวบน':'top3','2 ตัวล่าง':'up2'},inplace=True)

In [ ]:
lotto

,Date,1st,2 ตัวบน,top3,up2,3 ตัวหน้า,3 ตัวหน้า.1,3 ตัวล่าง,3 ตัวล่าง.1,weekday_dum,weekday
0,5/1/2024,980116,16,116,17,104,763,634,833,4,Wednesday
1,4/16/2024,943598,98,598,79,727,729,154,200,3,Tuesday
2,4/1/2024,803481,81,481,90,122,809,559,947,2,Monday
3,3/16/2024,997626,26,626,78,509,571,329,794,7,Saturday
4,3/1/2024,253603,3,603,79,900,975,382,703,6,Friday
...,...,...,...,...,...,...,...,...,...,...,...
716,5/16/1994,2071764,64,764,29,93,544,535,746,2,Monday
717,5/2/1994,3072877,77,877,69,535,317,170,521,2,Monday
718,4/16/1994,7893913,13,913,21,785,982,151,594,7,Saturday
719,4/1/1994,4269815,15,815,29,675,35,704,868,6,Friday


In [ ]:
ls= lotto.iloc[:,9:11]

In [ ]:
lu= lotto.iloc[:,0:5]

In [ ]:
lt = pd.concat([lu, ls], axis=1)

In [ ]:
lt

,Date,1st,2 ตัวบน,top3,up2,weekday_dum,weekday
0,5/1/2024,980116,16,116,17,4,Wednesday
1,4/16/2024,943598,98,598,79,3,Tuesday
2,4/1/2024,803481,81,481,90,2,Monday
3,3/16/2024,997626,26,626,78,7,Saturday
4,3/1/2024,253603,3,603,79,6,Friday
...,...,...,...,...,...,...,...
716,5/16/1994,2071764,64,764,29,2,Monday
717,5/2/1994,3072877,77,877,69,2,Monday
718,4/16/1994,7893913,13,913,21,7,Saturday
719,4/1/1994,4269815,15,815,29,6,Friday


In [ ]:
lt = lt.drop(columns=['2 ตัวบน'])

In [ ]:
lt

,Date,1st,top3,up2,weekday_dum,weekday
0,5/1/2024,980116,116,17,4,Wednesday
1,4/16/2024,943598,598,79,3,Tuesday
2,4/1/2024,803481,481,90,2,Monday
3,3/16/2024,997626,626,78,7,Saturday
4,3/1/2024,253603,603,79,6,Friday
...,...,...,...,...,...,...
716,5/16/1994,2071764,764,29,2,Monday
717,5/2/1994,3072877,877,69,2,Monday
718,4/16/1994,7893913,913,21,7,Saturday
719,4/1/1994,4269815,815,29,6,Friday


In [ ]:
lt.dtypes

Date           object
1st             int64
top3            int64
up2             int64
weekday_dum     int64
weekday        object
dtype: object

In [ ]:
more = lt[lt['1st'] >= 999999]
more.count()

Date           20
1st            20
top3           20
up2            20
weekday_dum    20
weekday        20
dtype: int64

In [ ]:
lt = lt[lt['1st'] <= 999999]

In [ ]:
lt['up2'] = lt['up2'].apply(lambda x: '{:02}'.format(x))

In [ ]:
lt['top3'] = lt['top3'].apply(lambda x: '{:03}'.format(x))

In [ ]:
lt['Year'] = pd.to_datetime(lt['Date']).dt.year
lt['Month'] = pd.to_datetime(lt['Date']).dt.month
lt['Day'] = pd.to_datetime(lt['Date']).dt.day

In [ ]:
lt_1 = lt

In [ ]:
lt_1['up2'] = lt_1['up2'].astype(str)

# Split each value in the 'digi' column into its individual digits
lt_1['Tens digit'] = lt_1['up2'].apply(lambda x: int(x[0]))
lt_1['Units digit'] = lt_1['up2'].apply(lambda x: int(x[1]) if len(x) > 1 else 0)

In [ ]:
lt_1['top3'] = lt_1['top3'].astype(str)

# Split each value in the 'digi' column into its individual digits
lt_1['c3_r1'] = lt_1['top3'].apply(lambda x: int(x[0]))
lt_1['c3_r2'] = lt_1['top3'].apply(lambda x: int(x[1]))
lt_1['c3_r3'] = lt_1['top3'].apply(lambda x: int(x[2]))

In [ ]:
lt_1

,Date,1st,top3,up2,weekday_dum,weekday,Year,Month,Day,Tens digit,Units digit,c3_r1,c3_r2,c3_r3
0,2024-05-01,980116,116,17,4,Wednesday,2024,5,1,1,7,1,1,6
1,2024-04-16,943598,598,79,3,Tuesday,2024,4,16,7,9,5,9,8
2,2024-04-01,803481,481,90,2,Monday,2024,4,1,9,0,4,8,1
3,2024-03-16,997626,626,78,7,Saturday,2024,3,16,7,8,6,2,6
4,2024-03-01,253603,603,79,6,Friday,2024,3,1,7,9,6,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,1995-03-16,646847,847,67,5,Thursday,1995,3,16,6,7,8,4,7
697,1995-03-01,411454,454,71,4,Wednesday,1995,3,1,7,1,4,5,4
698,1995-02-16,869070,070,95,5,Thursday,1995,2,16,9,5,0,7,0
699,1995-02-01,198162,162,48,4,Wednesday,1995,2,1,4,8,1,6,2
